In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from re import sub
from decimal import Decimal

#### First Look

In [3]:
df = pd.read_excel("Data\data.xlsx")

In [4]:
df.head()

,Title,Authors,Price,Language,NumOfPages,CostumerRatings,NumOfReviews,BookLink,Comments,search_word
0,Building Analytics Teams: Harnessing analytics...,John K. Thompson,$35.99,English,394.0,"4.5 out of 5 ['73%', '11%', '10%', '2%', '4%']",119 global ratings,https://www.amazon.com//gp/slredirect/picassoR...,['I have been doing analytics directly for 3 y...,Data Analytics
1,SQL QuickStart Guide: The Simplified Beginner'...,Walter Shields,$22.49,English,249.0,"4.6 out of 5 ['72%', '18%', '7%', '1%', '2%']","1,368 global ratings",https://www.amazon.com//gp/slredirect/picassoR...,"['This book was easy to follow and a great ""Qu...",Data Analytics
2,"Data Analytics, Data Visualization & Communica...",Elizabeth Clarke,$32.12,English,528.0,"4.6 out of 5 ['83%', '11%', '0%', '0%', '7%']",22 global ratings,https://www.amazon.com//gp/slredirect/picassoR...,['Don’t have any of the originals…not a proble...,Data Analytics
3,Hands-On Data Analysis with Pandas: A Python d...,Stefanie Molin,$35.67,English,788.0,"4.3 out of 5 ['65%', '18%', '7%', '3%', '6%']",72 global ratings,https://www.amazon.com//gp/slredirect/picassoR...,['This is a killer book for the Python and dat...,Data Analytics
4,Blueprints for Text Analytics Using Python: Ma...,Christian Winkler Jens Albrecht,$48.99,English,424.0,"4.8 out of 5 ['82%', '18%', '0%', '0%', '0%']",36 global ratings,https://www.amazon.com//Blueprints-Text-Analyt...,"[""I will start with a big thanks to the book a...",Data Analytics


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1093 non-null   object 
 1   Authors          1093 non-null   object 
 2   Price            1093 non-null   object 
 3   Language         1093 non-null   object 
 4   NumOfPages       1006 non-null   float64
 5   CostumerRatings  1093 non-null   object 
 6   NumOfReviews     1014 non-null   object 
 7   BookLink         1093 non-null   object 
 8   Comments         1093 non-null   object 
 9   search_word      1093 non-null   object 
dtypes: float64(1), object(9)
memory usage: 85.5+ KB


In [6]:
df.isnull().sum()

Title               0
Authors             0
Price               0
Language            0
NumOfPages         87
CostumerRatings     0
NumOfReviews       79
BookLink            0
Comments            0
search_word         0
dtype: int64

In [7]:
# let's firstly start by dropping duplicate records
print("Number of books before droping duplicate records: {}".format(df.shape[0]))
df = df.drop_duplicates(subset="Title")
print("Number of books after droping duplicate records: {}".format(df.shape[0]))

Number of books before droping duplicate records: 1093
Number of books after droping duplicate records: 733


In [8]:
# before cleaning each column separately, I want to select strings with length less than 4 as they are potential be missing values
print("Number of Potential Missing Values")
for column in list(df[["Title", "Authors", "Price", "Language", "CostumerRatings", "NumOfReviews", "BookLink"]].columns):
    n_missing = df.loc[df[column].str.len() < 4].shape[0]
    print(f"Column {column}: {n_missing}")

Number of Potential Missing Values
Column Title: 0
Column Authors: 0
Column Price: 0
Column Language: 0
Column CostumerRatings: 61
Column NumOfReviews: 0
Column BookLink: 0


#### Clean each columns

**Plan**
- Remove any needless strings;
- Remove outliers;
- Replace scrapping leftovers ("", or []) with missing values;
- Leave only English Language books;
- Extract information about Total Number of Reviews, Average Costumer Ratings and star proportions from CostumerRatings and NumeOfReviews;
- Separate comments from each other;
- Set appropriate data types for variables.

##### Price

In [9]:
# there should not be records without '$' sign. check if there is any
print("Number of records without '$' sign: {}".format(df[~df["Price"].str.startswith("$")].shape[0]))
# there should not be records with length more than 7 ("$***.**") as it's unlikely that books price is more than $1000,00
print("Number of records with length: {}".format(df.loc[df["Price"].str.len() > 7].shape[0]))

Number of records without '$' sign: 47
Number of records with length: 93


In [10]:
df[~df["Price"].str.startswith("$")].iloc[:3,:]

,Title,Authors,Price,Language,NumOfPages,CostumerRatings,NumOfReviews,BookLink,Comments,search_word
51,Data Analytics in Healthcare Research: Tools a...,Marc and Sandefer,Only 7 left in stock - order soon.,English,344.0,"4.7 out of 5 ['82%', '8%', '7%', '3%', '0%']",45 global ratings,https://www.amazon.com//Data-Analytics-Healthc...,"[""Interesting case studies and easy to read. I...",Data Analytics
77,Fundamentals of Machine Learning for Predictiv...,John D. Kelleher Brian Mac Namee,Only 16 left in stock (more on the way).,English,856.0,"4.9 out of 5 ['90%', '10%', '0%', '0%', '0%']",54 global ratings,https://www.amazon.com//Fundamentals-Machine-L...,['I have IT background as a Python Software De...,Data Analytics
137,Marketing Strategy: Based on First Principles ...,Shrihari Sridhar,Only 20 left in stock - order soon.,English,350.0,"3.9 out of 5 ['67%', '5%', '5%', '4%', '19%']",32 global ratings,https://www.amazon.com//Marketing-Strategy-Bas...,"[""I have been using the first edition to teach...",Data Analytics


In [11]:
# replace this values with np.nan
df.loc[~df["Price"].str.startswith("$"), "Price"] = np.nan

In [12]:
df.loc[df["Price"].str.len() > 7]["Price"].head(5)

21    $5.00\n               \n\n                Save...
41    $13.19\n               \n\n                Sav...
58    $2.99\n               \n\n                Save...
66    $29.99\n               \n\n                Sav...
76    $4.99\n               \n\n                Save...
Name: Price, dtype: object

In [13]:
# remove any character after first occurence of \n symbol
df.loc[(df["Price"].str.len() > 7), "Price"] =  df.loc[df["Price"].str.len() > 7]["Price"].apply(lambda x: x[: x.find("\n")])

In [14]:
# remove $ sign from prices and turn them into float type
df.loc[df["Price"].notnull(), "Price"] = df.loc[df["Price"].notnull(), "Price"].apply(lambda x: float(sub(r'[^\d.]', '', x)))

In [15]:
df.rename(columns={"Price": "Price ($)"}, inplace=True)
df["Price ($)"] = df["Price ($)"].astype("float")

##### Language

In [16]:
df.Language.value_counts()

English    732
Spanish      1
Name: Language, dtype: int64

In [17]:
df[df["Language"]=="Spanish"]

,Title,Authors,Price ($),Language,NumOfPages,CostumerRatings,NumOfReviews,BookLink,Comments,search_word
1058,The Hundred-Page Machine Learning Book en espa...,Andriy Burkov,43.88,Spanish,186.0,"4.4 out of 5 ['77%', '7%', '0%', '8%', '8%']",19 global ratings,https://www.amazon.com//Hundred-Page-Machine-L...,['Me agradó...estoy aprendiendo bastante de él...,Machine Learning


In [18]:
# as there is only one Spanish book, so let's remove it 
df = df.loc[df["Language"]=="English"]

##### CostumerRatings & NumOfReviews

In [19]:
# exrtact average review from column "CostumerRatings"
df["avg_review"] = df["CostumerRatings"].apply(lambda x: x[:3])
df["avg_review"] = df["avg_review"].str.replace("o", "").str.strip()

In [20]:
review_df = df["CostumerRatings"].str.split(",", expand=True)
review_df.columns = ["5_star", "4_star", "3_star", "2_star", "1_star"]

In [21]:
review_df.head(3)

,5_star,4_star,3_star,2_star,1_star
0,4.5 out of 5 ['73%','11%','10%','2%','4%']
1,4.6 out of 5 ['72%','18%','7%','1%','2%']
2,4.6 out of 5 ['83%','11%','0%','0%','7%']


In [22]:
# extract percentages from columns
review_df["5_star"] = review_df["5_star"].apply(lambda x: x[x.find("'")+1 : x.find("'", x.find("'")+1)])
for column in ["4_star", "3_star", "2_star", "1_star"]:
    review_df[column] = review_df[column].str.replace("'", "")
review_df["1_star"] = review_df["1_star"].str.replace("]", "", regex=False)

In [23]:
review_df.head(3)

,5_star,4_star,3_star,2_star,1_star
0,73%,11%,10%,2%,4%
1,72%,18%,7%,1%,2%
2,83%,11%,0%,0%,7%


In [24]:
df[["5_star", "4_star", "3_star", "2_star", "1_star"]] = review_df.values
# drop CostumerRatings as I extracted all needed information
df.drop(["CostumerRatings"], axis=1, inplace=True) 

##### NumOfReviews

In [25]:
df.NumOfReviews.head(3)

0      119 global ratings
1    1,368 global ratings
2       22 global ratings
Name: NumOfReviews, dtype: object

In [26]:
# remove any character after first occurence of " g" string
df.loc[df["NumOfReviews"].notnull(), "NumOfReviews"] = df.loc[df["NumOfReviews"].notnull(), "NumOfReviews"].apply(lambda x: x[:x.find(" g")])

In [27]:
df.NumOfReviews.head(3)

0      119
1    1,368
2       22
Name: NumOfReviews, dtype: object

##### Comments

In [28]:
# "\\n\\n\\n\\n\\n\\n\\" separates comments to each other, so I'll split via it to get single comment in each column
print("Maximum number of comments for signle book is: {}".format(df["Comments"].apply(lambda x: len(x.split("\\n\\n\\n\\n\\n\\n\\"))).max()))

Maximum number of comments for signle book is: 15


In [29]:
# prepare new dataframe to separate comments
new_df = pd.DataFrame(np.nan, index=[i for i in range(732)], columns=["com1","com2","com3","com4","com5","com6","com7","com8",
                                                                      "com9","com10","com11","com12","com13","com14","com15"])

In [30]:
# append comments into dataframe
for row_index, comments in enumerate(list(df["Comments"].apply(lambda x: x.split("\\n\\n\\n\\n\\n\\n\\")))):
    num_of_coms = len(comments)
    new_df.iloc[row_index, 0:num_of_coms] = comments

In [31]:
# try to remove 'Read more, ' string from columns
for column in ["com2","com3","com4","com5","com6","com7","com8","com9","com10","com11","com12","com13","com14","com15"]:
    new_df.loc[new_df[column].notnull(), column] = new_df.loc[new_df[column].notnull(), column].apply(lambda x: x[x.find("Read more")+11:])

In [32]:
# some more cleaning
for replace_symbol in ["n\\", "[", "]","\\", "n          ", "   "]:
    for column in ["com1","com2","com3","com4","com5","com6","com7","com8","com9","com10","com11","com12","com13","com14","com15"]:
        new_df.loc[new_df[column].notnull(), column] = new_df.loc[new_df[column].notnull(), column].apply(lambda x: x.replace(replace_symbol,""))

In [33]:
new_df.head(2)

,com1,com2,com3,com4,com5,com6,com7,com8,com9,com10,com11,com12,com13,com14,com15
0,'I have been doing analytics directly for 3 ye...,"'Some good ideas based on authors experience,...","""In a field as specialized as analytics, incl...","""This book was so inspiring, and I got so man...","'In this overly hyped field, organizations ar...",'John Thompson’s Building Analytics Teams is ...,'I have read this book from the very privileg...,'I highly recommend this book to anyone worki...,'Waste of time and money. Any one who has som...,'This book is not only a technical pleasant a...,'John has struck a blow for analytics teams a...,,NaN,NaN,NaN
1,"'This book was easy to follow and a great ""Qui...",'I have gone through five chapters of ten.. I...,'This book was exactly what I needed. Written...,'This is one of the best books of all times a...,'Book is missing some pages. It has reference...,'The free promotional items the company offer...,"'On the positive side, there are good, simple...","'Poorly written, lack of sql content deemed a...","""Simple to follow along and well-written. I w...","'Very Very basic, very short. Find something ...",'Excellent book for beginners,,NaN,NaN,NaN


In [34]:
df.reset_index(drop=True, inplace=True)
df.shape, new_df.shape

((732, 15), (732, 15))

In [35]:
df = df.join(new_df)
df.drop(["Comments"], axis=1, inplace=True) 

In [36]:
df["NumOfReviews"] = df["NumOfReviews"].str.replace(",", "").astype("float")

In [37]:
df.iloc[0:5, :10]

,Title,Authors,Price ($),Language,NumOfPages,NumOfReviews,BookLink,search_word,avg_review,5_star
0,Building Analytics Teams: Harnessing analytics...,John K. Thompson,35.99,English,394.0,119.0,https://www.amazon.com//gp/slredirect/picassoR...,Data Analytics,4.5,73%
1,SQL QuickStart Guide: The Simplified Beginner'...,Walter Shields,22.49,English,249.0,1368.0,https://www.amazon.com//gp/slredirect/picassoR...,Data Analytics,4.6,72%
2,"Data Analytics, Data Visualization & Communica...",Elizabeth Clarke,32.12,English,528.0,22.0,https://www.amazon.com//gp/slredirect/picassoR...,Data Analytics,4.6,83%
3,Hands-On Data Analysis with Pandas: A Python d...,Stefanie Molin,35.67,English,788.0,72.0,https://www.amazon.com//gp/slredirect/picassoR...,Data Analytics,4.3,65%
4,Blueprints for Text Analytics Using Python: Ma...,Christian Winkler Jens Albrecht,48.99,English,424.0,36.0,https://www.amazon.com//Blueprints-Text-Analyt...,Data Analytics,4.8,82%


In [38]:
df.to_excel("Data\cleaned_data.xlsx", index=False)